In [ ]:
import os
import re
import pandas as pd
import copy

In [ ]:
"""
Чтение файла и разбивка на список строк
input: расположение файла
output: список строк файла
"""

def read_file(file_directory):
    with open(file_directory, 'r', encoding='utf-8') as file:
        res_text_list = file.read().split('\n|\n')
        return res_text_list

In [ ]:
"""
Проверка ячейки строки на предмет наличия тегов, указывающих на стоимость
input: ячейка строки для проверки
output: статус проверки (True - содержит указание на стоимость, False - нет)
"""

def isPriceName(element):
    status = False
    if 'стоимост' in element.lower() or 'итог' in element.lower() or 'сумм' in element.lower():
        status = True
    return status

In [ ]:
"""
Поиск пар значений "заголовок - строка с sd или id"
input:текст из csv-файла, номер sd, номер id
output: результирующая строка
"""

def search_price(element, sd, qw):
    indexies = []
    lines = element.split('\n')
    first_line = lines[0]
    second_line = lines[1]
    splited_first_line = first_line.split(';')
    splited_second_line = second_line.split(';')
    fl,sl = [],[]
    for index, fl_el in enumerate(splited_first_line):
        if (isPriceName(fl_el)):
            indexies.append(index)
    print(indexies)
    for i in indexies:
        try:
            fl.append(splited_first_line[i])
            sl.append(splited_second_line[i])
        except:
            print("Error")
            continue
    res_string='SD_{0};QW_{1};{2}\n{3};{4};{5}\n'.format(sd,qw,';'.join(fl),sd,qw,';'.join(sl))
    print(res_string)
    return res_string

In [ ]:
"""
Поиск в строке sd или id
input:номер id или sd, строка в которой необходимо произвести поиск
output: статус проверки строки (присутствует sd или id в строке)
"""

def search_id_qw(sd, line):
    status = False
    if sd=="nan":
        return status
    if len(re.findall(r';\s*(sd|)'+str(sd)+'\s*(;|)',line.lower()))>0:
        status  =True
    return status

In [ ]:
list_elements = read_file('nzya2_z1p1.csv')
df = pd.read_csv('nzya2_z1p1_files.csv',sep=';', encoding="utf-8")
copy_list_elements = copy.copy(list_elements)

In [ ]:
df.head()

In [ ]:
with open('price.csv','w') as file:
    for index,row in df.iterrows():
        for i,element in enumerate(list_elements):
            first_col_val = str(row[1])
            sec_col_val = str(row[2])[:-2] # срез строки (если начинается с sd...)
            if 'nan' in sec_col_val:
                sec_col_val="1253463745687676532574p"
            if search_id_qw(first_col_val,element) or search_id_qw(sec_col_val,element):
                file.write(search_price(element,first_col_val,sec_col_val))
                del copy_list_elements[i]
                print("Gotcha"+str(len(copy_list_elements)))
                break
        list_elements = copy.copy(copy_list_elements)